In [1]:
import gc
import re
import traceback
import os
import sys
from tqdm import tqdm

In [2]:
import pickle

In [3]:
with open("/home/jrj/postgraduate/Symbolic/Backdoor/result/verified/vul_result.pkl", "rb") as fr:
    res = pickle.load(fr)
for d in res:
    res[d] = dict(sorted(res[d].items(), key=lambda x:x[1], reverse=True))

In [8]:
s_res = ""
for d in res:
    content = f"{str(d)}:"
    for t in res[d]:
        content += f"{str(t)}({str(res[d][t])}),\\ "
    s_res += f"{content}\n"
print(s_res)

In [3]:
def all_path(dirname, filter_list):
    result = []  # 所有的文件
    # filter = [".wav"]
    for maindir, subdir, file_name_list in os.walk(dirname):
        for filename in file_name_list:
            apath = os.path.join(maindir, filename)  # 合并成一个完整路径
            ext = os.path.splitext(apath)[1]
            if ext in filter_list:
                result.append(apath)
    return result

In [ ]:
src_root_path = "/home/jrj/postgraduate/backdoor_data/dataset/verified/modified"
folders = os.listdir(src_root_path)
tmp_result = {}
for folder in folders:
    src_folder_path = os.path.join(src_root_path, folder)
    sols = all_path(src_folder_path, [".sol"])
    tmp_result[folder] = len(sols)
 

In [13]:
outs = all_path("/home/jrj/postgraduate/PiedPiperBackdoor/tools/analyser",[".out"])
result = {}
for file in outs:
    filename = os.path.basename(file).split("_")
    b_type = filename[2]
    batched = filename[1]
    total = 0.0
    with open(file, "r") as fr:
        content = fr.readlines()
    for line in content:
        l = line.split(",")
        if len(l) < 3: continue
        time_1, time_2 = float(l[1]), float(l[2])
        total += time_1+time_2
    result[b_type] = result.get(b_type, 0) + (total/len(content))      

In [3]:
import glob
file_pattern = "*.sol"

In [10]:
path = "/home/jrj/postgraduate/backdoor_data/bsc_contracts"
folders = os.listdir(path)
# files = all_path(path, ['.sol'])

In [12]:
from fuzzywuzzy import fuzz
from backdoor.utils.node_utils import is_mapping_type, is_number_type
from backdoor.state.solidity_info import SolidityInfo
from backdoor.utils.preprocess import SolFile
from slither import Slither
from scripts.process_dataset import get_solc_version

candidates = []
for folder in tqdm(folders):
    sols = all_path(os.path.join(path, folder), [".sol"])
    if len(sols) != 1: continue
    file = sols[0]
    try:
        solc_version = get_solc_version(file)
        solc_path = f"/home/jrj/.solc-select/artifacts/solc-{solc_version}"
        sol_file = SolidityInfo(path, solc_path)
        file_flag = False
        for c in sol_file.get_contracts_derived():
            if file_flag: break
            has_balances = False
            for state_var in c.state_variables:
                if is_mapping_type(state_var.type):
                    if is_number_type(state_var.type.type_to):
                        if fuzz.partial_ratio(state_var.name, "balances") >= 85:
                            has_balances = True
                            break
            if not has_balances:
                file_flag = True
                candidates.append(file)
    except:
        continue
        

In [11]:
eth = all_path("/home/jrj/postgraduate/Symbolic/Backdoor/dataset/eth", [".sol"])
bsc = all_path("/home/jrj/postgraduate/Symbolic/Backdoor/dataset/bsc", [".sol"])
total = eth + bsc
total = [os.path.basename(t) for t in total]

In [12]:
r_v2 = all_path("/home/jrj/postgraduate/Symbolic/Backdoor/dataset/real_world_v2", [".sol"])
r_selectedv2 = all_path("/home/jrj/postgraduate/Symbolic/Backdoor/dataset/real_world_selected_v2", [".sol"])
r_v2 = set([os.path.basename(r) for r in r_v2])
r_selectedv2 = set([os.path.basename(rs) for rs in r_selectedv2])
total = set(total)

In [13]:
len(r_v2.difference(total))

In [2]:
print("123")